In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os
import numpy as np
import pandas as pd
import sys
from glob import glob
from tqdm import tqdm
REPO_DIR = "/Users/mdorosan/Desktop/2024/elpha-singhealth-fh"
sys.path.append(REPO_DIR)

import utils.PATHS as PATHS
import utils.utils as utils
# import utils.emr_utils as emr_utils
# import utils.load_utils as load_utils

## Notes

The following were observed in the exploration below:
* Potentially useful columns:
  - `Lab Resulted Order Test Description` is more intuitive in mapping to a the `Result value` column
  - `Lab Test Description` is broader. Such as it may consist of different `Lab Resulted Order Test Description`
* Reliable datetime columns (for Index Date of highest LDL-C), ordered by most suitable:
  - `Specimen Received Date`
  - `Specimen Collection Date`
  - `Admit Date`
* **Issue**: Lab Test Shortext Descriptions provided by collaborators dont match with SGH vocabulary. Resolve with clinician input.

In [9]:
lab_fp_list = glob(os.path.join(PATHS.LABS, "*.csv"))

In [10]:
print(lab_fp_list[0])
test = pd.read_csv(lab_fp_list[0])
test.info()

/Users/mdorosan/Desktop/2024/datasets/elpha-fh/1 Lab General/2024-03-25_Lab_General_Subject_FH_Table_1_(SGH_2017-05May).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712257 entries, 0 to 712256
Data columns (total 25 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Institution Code                         712257 non-null  object 
 1   Patient ID                               712257 non-null  object 
 2   Gender                                   712257 non-null  object 
 3   Race                                     712257 non-null  object 
 4   Date of Birth                            712257 non-null  object 
 5   Nationality                              712257 non-null  object 
 6   Country of Residence                     57680 non-null   object 
 7   Resident Indicator                       712257 non-null  object 
 8   Case No                                  712257 no

In [11]:
# lab results columns
lab_cols = [
    "Lab Test Code",
    "Lab Test Description",
    "Lab Test Type",
    "Lab Test Type (KKH Only)",
    "Lab Test Display",
    "Result Value"
]
test[lab_cols]

,Lab Test Code,Lab Test Description,Lab Test Type,Lab Test Type (KKH Only),Lab Test Display,Result Value
0,A1,"UE,BI,CRE,GLU",P,NaN,NaN,17.8
1,A1,"UE,BI,CRE,GLU",P,NaN,NaN,111
2,A1,"UE,BI,CRE,GLU",P,NaN,NaN,126
3,A1,"UE,BI,CRE,GLU",P,NaN,NaN,3.8
4,A1,"UE,BI,CRE,GLU",P,NaN,NaN,5.6
...,...,...,...,...,...,...
712252,UME,URINE MICROSCOPIC EXAM,P,NaN,NaN,5
712253,UME,URINE MICROSCOPIC EXAM,P,NaN,NaN,FEW
712254,UME,URINE MICROSCOPIC EXAM,P,NaN,NaN,NIL
712255,UME,URINE MICROSCOPIC EXAM,P,NaN,NaN,33


In [12]:
# ROTD per TD
test[["Lab Test Description", "Lab Resulted Order Test Description", "Result Value"]].groupby("Lab Test Description")["Lab Resulted Order Test Description"].apply(set)

Lab Test Description
(CK,MB,TNT)                 {CREATINE KINASE, CREATINE KINASE MB (MASS), (...
(PAPP-A,FBHCG)                   {PREG ASSOC PLASMA PROTEIN A, FREE BETA HCG}
(Tg,TgAb)                           {THYROGLOBULIN, THYROGLOBULIN ANTIBODIES}
24HR UR FREE CORTISOL(P)    {24HR URINE VOLUME, 24H UR FREE CORTISOL(T), F...
24HR URINE CALCIUM          {CALCIUM,URINE, 24HR URINE CALCIUM, 24HR URINE...
                                                  ...                        
ZINC,SERUM                                                       {ZINC,SERUM}
pCO2,POCT                                                         {pCO2,POCT}
pH,BLOOD POCT                                                 {pH,BLOOD POCT}
pO2,POCT                                                           {pO2,POCT}
tHb,POCT                                                           {tHb,POCT}
Name: Lab Resulted Order Test Description, Length: 599, dtype: object

In [13]:
# TD per ROTD : See notes on findings
test[["Lab Test Description", "Lab Resulted Order Test Description", "Result Value"]].groupby("Lab Resulted Order Test Description")["Lab Test Description"].apply(set)

Lab Resulted Order Test Description
%HEAT STABLE ALP                    {HEAT STABLE ALP}
(CK,MB,TNT)                             {(CK,MB,TNT)}
24H UR FREE CORTISOL(T)    {24HR UR FREE CORTISOL(P)}
24HR CU URINE VOL                 {24HR URINE COPPER}
24HR MG URINE VOLUME           {24HR URINE MAGNESIUM}
                                      ...            
pH,BLOOD                                {BLOOD GASES}
pH,BLOOD POCT                         {pH,BLOOD POCT}
pO2                                     {BLOOD GASES}
pO2,POCT                                   {pO2,POCT}
tHb,POCT                                   {tHb,POCT}
Name: Lab Test Description, Length: 881, dtype: object

In [14]:
test["Lab Test Description"].value_counts()

Lab Test Description
FBC                               338076
GLUCOSE,POCT                       62903
UE,BI,CRE                          54822
UE,BI,CRE,GLU                      47006
LIVER PANEL (WITH GGT)             30049
                                   ...  
Coxiella burnetti CFT                  1
CD138 PLASMA CELL                      1
CARCINO EMBRYONIC ANTIGEN,FLUI         1
LACTATE,FLUID                          1
Mumps Ag                               1
Name: count, Length: 599, dtype: int64

In [15]:
other_lab_cols = [
    "Lab Test Code",
    "Lab Resulted Order Test Description",
    "Lab Resulted Order Test Type",    
    "Lab Resulted Order Test Type (KKH Only)",
    "Lab Resulted Order Test Display",
    "Result Value"    
]
test[other_lab_cols]

,Lab Test Code,Lab Resulted Order Test Description,Lab Resulted Order Test Type,Lab Resulted Order Test Type (KKH Only),Lab Resulted Order Test Display,Result Value
0,A1,BICARBONATE,T,NaN,NaN,17.8
1,A1,CHLORIDE,T,NaN,NaN,111
2,A1,CREATININE,T,NaN,NaN,126
3,A1,GLUCOSE,T,NaN,NaN,3.8
4,A1,POTASSIUM,T,NaN,NaN,5.6
...,...,...,...,...,...,...
712252,UME,EPITHELIAL CELL,T,NaN,NaN,5
712253,UME,MICRO-ORGANISMS,T,NaN,NaN,FEW
712254,UME,URINE CASTS,T,NaN,NaN,NIL
712255,UME,URINE RBC,T,NaN,NaN,33


In [16]:
test["Lab Resulted Order Test Description"].value_counts()

Lab Resulted Order Test Description
GLUCOSE,POCT                      62903
POTASSIUM                         17563
CREATININE                        17166
UREA                              16773
SODIUM                            16635
                                  ...  
CD79 BETA B-CELLS                     1
CRYOPRESERVATION                      1
AUTO AGGLUTINATION                    1
CARCINO EMBRYONIC ANTIGEN,FLUI        1
(CK,MB,TNT)                           1
Name: count, Length: 881, dtype: int64

In [17]:
# date columns
dt_cols = [
    "Result Comment Date",
    "Specimen Received Date",
    "Specimen Received Date.1",
    "Admit Date",
    "Specimen Collection Date",
    "Visit Date"
]
test[dt_cols]

,Result Comment Date,Specimen Received Date,Specimen Received Date.1,Admit Date,Specimen Collection Date,Visit Date
0,NaN,2017-05-03,2017-05-03,2017-05-03,2017-05-03,NaN
1,NaN,2017-05-03,2017-05-03,2017-05-03,2017-05-03,NaN
2,NaN,2017-05-03,2017-05-03,2017-05-03,2017-05-03,NaN
3,NaN,2017-05-03,2017-05-03,2017-05-03,2017-05-03,NaN
4,NaN,2017-05-03,2017-05-03,2017-05-03,2017-05-03,NaN
...,...,...,...,...,...,...
712252,NaN,2017-05-21,2017-05-21,2017-05-21,2017-05-21,NaN
712253,NaN,2017-05-21,2017-05-21,2017-05-21,2017-05-21,NaN
712254,NaN,2017-05-21,2017-05-21,2017-05-21,2017-05-21,NaN
712255,NaN,2017-05-21,2017-05-21,2017-05-21,2017-05-21,NaN


In [18]:
# check LDL-C Tests
(test.loc[test["Lab Resulted Order Test Description"].str.contains("LDL", case=False)]
 [["Patient ID", "Lab Resulted Order Test Description", "Result Value"]])["Lab Resulted Order Test Description"].unique()

array(['LDL-CHOLESTEROL,CALCULATED', 'CHOLESTEROL,TG,HDL,LDL',
       'LDL-CHOLESTEROL,DIRECT'], dtype=object)

In [19]:
# check LDL-C Tests
(test.loc[test["Lab Test Description"].str.contains("LDL", case=False)]
 [["Patient ID", "Lab Test Description", "Result Value"]])["Lab Test Description"].unique()

array(['CHOLESTEROL,TG,HDL,LDL', 'LDL-CHOLESTEROL,DIRECT'], dtype=object)

In [20]:
# # check values on secondary conditions related to hypocholesterolemia
# for i in utils.secondary_labs_st:
#     print(i.upper())
[i.lower() for i in utils.secondary_labs_st]

['tsh',
 'bilirubin, total',
 'protein, total urine',
 'protein tot u 24h',
 'creatinine',
 'creatinine, poct',
 'egfr',
 'hba1c',
 'hba1c, poct',
 'hba1c, rapid',
 'glucose, fasting']

In [21]:
# test orders dictionary
df_list = [pd.read_csv(path, low_memory=False) for path in tqdm(lab_fp_list)]
df = pd.concat(df_list, ignore_index=True)

100%|███████████████████████████████████████████| 48/48 [00:58<00:00,  1.22s/it]


In [22]:
# df.info()

In [23]:
# create vocabulary of test orders
test_orders_vocab = df["Lab Resulted Order Test Description"].unique().tolist()

In [24]:
[i for i in test_orders_vocab if "tsh" in i.lower()]

['TSH RECEPTOR ANTIBODY',
 'FREE THYROXINE,TSH',
 'TSH,CORD BLOOD',
 'TSH-CONGENITAL HYPOTHYROID SCR',
 'TSH RECEPTOR ANTIBODIES (TRAb)']

In [25]:
[i for i in test_orders_vocab if "bilirubin" in i.lower()]

['BILIRUBIN,TOTAL',
 'BILIRUBIN,DIRECT',
 'TOTAL BILIRUBIN,FLUID',
 'BILIRUBIN DIPSTIX',
 'BILIRUBIN,PAEDIATRIC',
 'BILIRUBIN DIRECT,FLUID']

In [26]:
[i for i in test_orders_vocab if "protein" in i.lower()]

['PROTEIN,TOTAL',
 'C REACTIVE PROTEIN',
 'PROTEIN,URINE',
 'URINE PROTEIN/CRE RATIO',
 'PROTEIN,24HR URINE',
 'ALPHAFOETO PROTEIN',
 'PROTEIN, TOTAL (PE)',
 'URINE PROTEIN ELECTROPHORESIS',
 'URINE TOTAL PROTEIN',
 'PROTEIN C',
 'PROTEIN S (TOTAL)',
 'URINE PROTEIN',
 'T.PROTEIN,CSF',
 'T.PROTEIN,FLUID',
 'SERUM PROTEIN ELECTROPHORESIS',
 'PROTEIN S-FREE',
 'PROTEIN DIPSTIX',
 'PROTEIN S (FUNCTIONAL)',
 'UR PROTEIN/CRE RATIO PROFILE',
 'PREG ASSOC PLASMA PROTEIN A',
 'GPI-PROTEIN',
 'ALPHAFOETO PROTEIN,FLUID',
 'APOLIPOPROTEIN A-1',
 'APOLIPOPROTEIN B',
 'PROTEIN.T/ALBUMIN',
 '24HR U PROTEIN ELECTROPHORESIS']

In [27]:
[i for i in test_orders_vocab if "urine" in i.lower() and "protein" in i.lower()]

['PROTEIN,URINE',
 'URINE PROTEIN/CRE RATIO',
 'PROTEIN,24HR URINE',
 'URINE PROTEIN ELECTROPHORESIS',
 'URINE TOTAL PROTEIN',
 'URINE PROTEIN']

In [28]:
[i for i in test_orders_vocab if "creatinine" in i.lower()]

['CREATININE',
 'CREATININE,URINE',
 'CREATININE,FLUID',
 '24HR URINE CREATININE',
 'CREATININE CLEARANCE',
 'CREATININE,UR (CALC)',
 'MICROALB/CREATININE',
 'GLUCOSE,CREATININE',
 'CREATININE CLEARANCE TEST',
 'VMA/CREATININE URINE',
 'ALBUMIN/CREATININE RATIO,URINE',
 'HVA/CREATININE URINE']

In [29]:
[i for i in test_orders_vocab if "egfr" in i.lower()]

['EGFR MUTATION DETECTION',
 'EGFR REAL TIME PCR',
 'MDRD eGFR',
 'CORRECTED CALC CG eGFR',
 'CKD-EPI eGFR',
 'RENAL PANEL WITH MDRD eGFR',
 'RENAL PANEL WITH CKD-EPI eGFR',
 'RENAL PANEL WITH CG eGFR',
 'CG eGFR',
 'BLOOD EGFR MUTATION ANALYSIS']

In [30]:
# efgr - Estimated glomerular filtration rate
[i for i in test_orders_vocab if "glomerular" in i.lower()]

[]

In [31]:
[i for i in test_orders_vocab if "hba1c" in i.lower()]

['HBA1C', 'HBA1C IFCC']

In [32]:
[i for i in test_orders_vocab if "hba" in i.lower()]

['HBA1C', 'HBA1C IFCC']

In [33]:
[i for i in test_orders_vocab if "glucose" in i.lower()]

['GLUCOSE',
 'GLUCOSE,POCT',
 'GLUCOSE,PLASMA FASTING',
 'GLUCOSE DIPSTIX',
 'GLUCOSE,PLASMA',
 'GLUCOSE,CSF',
 'GLUCOSE,FLUID',
 'GLUCOSE 120 MIN',
 'GLUCOSE FASTING',
 'URINE GLUCOSE',
 'GLUCOSE,CREATININE',
 'GLUCOSE 60 MIN',
 'GLUCOSE TOLERANCE 2 SPEC',
 'GLUCOSE TOLERANCE 3 SPEC']

In [34]:
[i for i in test_orders_vocab if "rapid" in i.lower()]

[]

In [35]:
[i for i in test_orders_vocab if "fasting" in i.lower()]

['GLUCOSE,PLASMA FASTING', 'GLUCOSE FASTING']

In [36]:
[i for i in test_orders_vocab if "poct" in i.lower() and "creatinine" in i.lower()]

[]

In [37]:
[i for i in test_orders_vocab if "poct" in i.lower()]

['GLUCOSE,POCT',
 'BE,POCT',
 'CALCIUM,IONIC POCT',
 'Hct,POCT',
 'POTASSIUM,POCT',
 'SODIUM,POCT',
 'pCO2,POCT',
 'pH,BLOOD POCT',
 'pO2,POCT',
 'O2 SAT,POCT',
 'STD BICARBONATE,POCT',
 'tHb,POCT',
 'CHLORIDE,POCT',
 'COHb,POCT',
 'HHb,POCT',
 'MetHb,POCT',
 'O2 Hb,POCT',
 'INR,POCT',
 'LACTATE,POCT']